In [10]:
from typing import Tuple
import pandas as pd
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt

# are the principal components of the covariance matrix the same as those of the correlation?

No! Why? Because non-uniform scaling is not angle-preserving.

Imagine that we have some clean, standardized data.
We calculate its covariance matrix and call it $\Omega$.
We then multiply each variable by the corresponding scalar specified in $\delta$
(think of this as converting from kg to lbs, or km to mi, or L to gal).
We calculate the new data's covariance matrix and call it $\Sigma$.
The data used to be "shaped" like a blob in $N$-space,
and we have deformed that blob;
In so doing, we have fundamentally changed its "explanation".
This is a pretty strong statement:
It's not just that the PC's are the same but their order has changed;
It's not just that the old PC's are still orthogonal to each other but are no longer the most "efficient" encoding;
It's not even just that you can get the new PC's from the old by applying $\Delta^{-1}$ to rectify the deformation.
The transformed old PC's are no longer guaranteed to be PC's or even orthogonal to each other,

raw data to transformed: X -> XDelta
cov: (XD)'XD = D'X'XD = DX'XD = DOD


You can see that the new eigenvectors won't be the same:
We know that $\Sigma = \Delta \Omega \Delta$,
so consider $v$ where $v$ is an eigenvector of $\Omega$, i.e.
$\Omega v = a v$ where $a$ is some scalar.
Now let's consider $\Sigma v = \Delta \Omega \Delta v$.
Immediately, we run into an issue:
We want to apply $\Omega$ to $v$, but
instead we're applying it to $\Delta v$.
Alright, let's remedy the issue:
We'll let ourselves relax a bit, and concede that $v$
might not be an eigenvector of $\Sigma$, but
surely $u := \Delta^{-1} v$ will be!
Let's try it out: We have,

$$ \Sigma u = \Delta \Omega \Delta (\Delta^{-1} v) $$
$$ = \Delta \Omega v $$
$$ = \Delta a v $$
$$ = a \Delta v ...$$
Dammit! We wanted something like $\Sigma u = b u = b \Delta^{-1} v$,
but instead of $\Delta^{-1}$, god laughed in our face and gave us the inverse.

So it's not trivial to go from one to the other.
Below, we'll build some visual intution as to why.

# setting

## correlation

In [36]:
# correlation
omega = 0.1
Omega = pd.DataFrame({
    "X0": {"X0": 1    , "X1": omega},
    "X1": {"X0": omega, "X1": 1    }
})
Omega

,X0,X1
X0,1.0,0.1
X1,0.1,1.0


## standard deviation

In [68]:
delta = pd.Series({"X0": 1, "X1": 2})
# diagonal square matrix with standard deviations along diagonal
Delta = pd.DataFrame(
    np.diag(delta),
index=delta.index, columns=delta.index)
Delta

,X0,X1
X0,1,0
X1,0,2


## covariance

In [38]:
Sigma = Delta @ Omega @ Delta
Sigma

,X0,X1
X0,1.0,0.2
X1,0.2,4.0


# eigendecompositions

In [69]:
def _eig(mat: pd.DataFrame) -> Tuple[pd.Series, pd.DataFrame]:
    """
    Eigendecompose `mat`, returning eigenvalues `W` and corresponding eigenvectors `V`,
    such that the eigenvector associated with eigenvalue `W[n]` is `V[:, n]`.
    """
    W, V = np.linalg.eig(mat)
    W = pd.Series(W)
    V = pd.DataFrame(V, index=mat.index)
    # sort in order of explained variance, then reorder v to match
    W = W.sort_values(ascending=False)
    V = V.reindex(columns=W.index)
    # the order it came out of `np.eig` is not meaningful, drop it
    W = W.reset_index(drop=True)
    V = V.T.reset_index(drop=True).T  # stupid hack, there is no `pd.DataFrame.reset_columns()`
    # make the column names more suggestive
    W = W.rename(index=lambda n: f"E{n}")
    V = V.rename(columns=lambda n: f"E{n}")
    # i hate vectors with negative heads, so if i find one, negate the entire vector
    sign_of_V_heads = np.sign(V.loc["X0", :])
    V = V.mul(sign_of_V_heads, axis="columns")
    return W, V

def eig(mat: pd.DataFrame) -> Tuple[pd.Series, pd.DataFrame]:
    """Like `eig()` but with side effects :)."""
    W, V = _eig(mat=mat)
    print(W)
    print()
    print(V)
    return W, V

## correlation

In [70]:
W_Omega, V_Omega = eig(mat=Omega)

E0    1.1
E1    0.9
dtype: float64

          E0        E1
X0  0.707107  0.707107
X1  0.707107 -0.707107


## covariance

In [71]:
W_Sigma, V_Sigma = eig(mat=Sigma)

E0    4.013275
E1    0.986725
dtype: float64

          E0        E1
X0  0.066227  0.997805
X1  0.997805 -0.066227


## entrywise ratio of cov PC's to corr PC's, and vice versa

In [72]:
V_Sigma / V_Omega

,E0,E1
X0,0.093659,1.411109
X1,1.411109,0.093659


In [73]:
V_Omega / V_Sigma

,E0,E1
X0,10.676975,0.708663
X1,0.708663,10.676975


# visualization